# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2928443938382,
 'week52high': 187.94,
 'week52low': 120.63,
 'week52highSplitAdjustOnly': 191.57,
 'week52lowSplitAdjustOnly': 120.82,
 'week52change': 0.334913923910575,
 'sharesOutstanding': 16869208644,
 'float': 0,
 'avg10Volume': 87106935,
 'avg30Volume': 107804917,
 'day200MovingAvg': 152.42,
 'day50MovingAvg': 172.83,
 'employees': 152520,
 'ttmEPS': 11.47,
 'ttmDividendRate': 0.8842059773242671,
 'dividendYield': 0.00505235375375405,
 'nextDividendDate': '',
 'exDividendDate': '2021-11-04',
 'nextEarningsDate': '2022-01-20',
 'peRatio': 15.44334461851454,
 'beta': 1.345283991868498,
 'maxChangePercent': 67.77064961883174,
 'year5ChangePercent': 5.266453015651572,
 'year2ChangePercent': 1.333621908179841,
 'year1ChangePercent': 0.3818490671069878,
 'ytdChangePercent': -0.03289603538553652,
 'month6ChangePercent': 0.21971854924444809,
 'month3ChangePercent': 0.21548838284938954,
 'month1ChangePercent': 0.04056085731466057,
 'day30ChangeP

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.3818490671069878

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

# for symbol_string in symbol_strings:
#     print(symbol_string)
    
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    #print(symbol_string.split(','))
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
        pd.Series(
        [
            symbol, 
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index=my_columns),
        ignore_index=True)

final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,146.950,0.227321,N/A
1,AAL,20.330,0.201808,N/A
2,AAP,239.150,0.493898,N/A
3,AAPL,178.560,0.38319,N/A
4,ABBV,139.500,0.349302,N/A
...,...,...,...,...
500,YUM,137.280,0.32305,N/A
501,ZBH,131.748,-0.192763,N/A
502,ZBRA,548.613,0.414663,N/A
503,ZION,71.750,0.47735,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_df.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_df = final_df[:50]
final_df.reset_index(inplace = True)
final_df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,80.160,2.295629,N/A
1,148,DVN,50.060,1.836265,N/A
2,174,F,24.690,1.782108,N/A
3,317,MRO,19.010,1.473242,N/A
4,299,MCHP,85.600,1.458241,N/A
5,195,FTNT,326.681,1.291788,N/A
6,175,FANG,124.995,1.286532,N/A
7,345,NVDA,282.290,1.27843,N/A
8,344,NUE,117.850,1.100344,N/A
9,173,EXR,213.370,1.098582,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    try: 
        float(portfolio_size)
    except ValueError:
        print('That is not a number')
        print('Please try again')
        portfolio_size = input('Enter the size of your portfolio:')
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:25000
25000


In [10]:
position_size = float(portfolio_size)/len(final_df.index)
for i in range(0, len(final_df)):
    final_df.loc[i, "Number of Shares to Buy"] = math.floor(position_size/final_df.loc[i, 'Price'])

final_df

C:\Users\Local User\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,80.160,2.295629,6
1,148,DVN,50.060,1.836265,9
2,174,F,24.690,1.782108,20
3,317,MRO,19.010,1.473242,26
4,299,MCHP,85.600,1.458241,5
5,195,FTNT,326.681,1.291788,1
6,175,FANG,124.995,1.286532,4
7,345,NVDA,282.290,1.27843,1
8,344,NUE,117.850,1.100344,4
9,173,EXR,213.370,1.098582,2


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [11]:
hqm_columns = [
    "Ticker",
    "Price",
    "Number of Shares to Buy",
    "One-Year Price Return",
    "One-Year Return Percentile",
    "Six-Month Price Return",
    "Six-Month Return Percentile",
    "Three-Month Price Return",
    "Three-Month Return Percentile",
    "One-Month Price Return",
    "One-Month Return Percentile", 
    "HQM Score"
]

hqm_df = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                "N/A",
                data[symbol]["stats"]["year1ChangePercent"],
                "N/A",
                data[symbol]["stats"]["month6ChangePercent"],
                "N/A",
                 data[symbol]["stats"]["month3ChangePercent"],
                "N/A",
                 data[symbol]["stats"]["month1ChangePercent"],
                "N/A",
                "N/A"],
                index = hqm_columns),
            ignore_index = True
            )
        
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,151.485,N/A,0.230792,N/A,0.003598,N/A,-0.02863,N/A,-0.014412,N/A,N/A
1,AAL,19.590,N/A,0.19686,N/A,-0.12001,N/A,-0.097372,N/A,0.036932,N/A,N/A
2,AAP,249.582,N/A,0.494325,N/A,0.176768,N/A,0.151211,N/A,0.055483,N/A,N/A
3,AAPL,180.430,N/A,0.380283,N/A,0.223255,N/A,0.222448,N/A,0.040435,N/A,N/A
4,ABBV,141.400,N/A,0.357486,N/A,0.203052,N/A,0.255466,N/A,0.114918,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,137.040,N/A,0.308795,N/A,0.195558,N/A,0.116829,N/A,0.065474,N/A,N/A
501,ZBH,132.140,N/A,-0.201153,N/A,-0.213168,N/A,-0.120879,N/A,0.028754,N/A,N/A
502,ZBRA,561.812,N/A,0.408689,N/A,0.020672,N/A,0.096454,N/A,-0.051653,N/A,N/A
503,ZION,71.040,N/A,0.461176,N/A,0.362315,N/A,0.105308,N/A,0.079699,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [12]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]
hqm_df.dropna(inplace=True)
for row in hqm_df.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_df.loc[row, percentile_col] = stats.percentileofscore(hqm_df[change_col], hqm_df.loc[row, change_col])/100

# Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(hqm_df[percentile_col])

#Print the entire DataFrame    
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,151.485,N/A,0.230792,0.469062,0.003598,0.289421,-0.02863,0.157685,-0.014412,0.167665,N/A
1,AAL,19.590,N/A,0.19686,0.417166,-0.12001,0.08982,-0.097372,0.06986,0.036932,0.421158,N/A
2,AAP,249.582,N/A,0.494325,0.832335,0.176768,0.738523,0.151211,0.770459,0.055483,0.560878,N/A
3,AAPL,180.430,N/A,0.380283,0.692615,0.223255,0.840319,0.222448,0.91018,0.040435,0.449102,N/A
4,ABBV,141.400,N/A,0.357486,0.672655,0.203052,0.794411,0.255466,0.9501,0.114918,0.908184,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,137.040,N/A,0.308795,0.602794,0.195558,0.776447,0.116829,0.654691,0.065474,0.660679,N/A
501,ZBH,132.140,N/A,-0.201153,0.023952,-0.213168,0.027944,-0.120879,0.043912,0.028754,0.379242,N/A
502,ZBRA,561.812,N/A,0.408689,0.734531,0.020672,0.325349,0.096454,0.59481,-0.051653,0.055888,N/A
503,ZION,71.040,N/A,0.461176,0.802395,0.362315,0.96008,0.105308,0.61477,0.079699,0.738523,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [13]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
    hqm_df.loc[row, "HQM Score"] = mean(momentum_percentiles)
    
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,151.485,N/A,0.230792,0.469062,0.003598,0.289421,-0.02863,0.157685,-0.014412,0.167665,0.270958
1,AAL,19.590,N/A,0.19686,0.417166,-0.12001,0.08982,-0.097372,0.06986,0.036932,0.421158,0.249501
2,AAP,249.582,N/A,0.494325,0.832335,0.176768,0.738523,0.151211,0.770459,0.055483,0.560878,0.725549
3,AAPL,180.430,N/A,0.380283,0.692615,0.223255,0.840319,0.222448,0.91018,0.040435,0.449102,0.723054
4,ABBV,141.400,N/A,0.357486,0.672655,0.203052,0.794411,0.255466,0.9501,0.114918,0.908184,0.831337
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,137.040,N/A,0.308795,0.602794,0.195558,0.776447,0.116829,0.654691,0.065474,0.660679,0.673653
501,ZBH,132.140,N/A,-0.201153,0.023952,-0.213168,0.027944,-0.120879,0.043912,0.028754,0.379242,0.118762
502,ZBRA,561.812,N/A,0.408689,0.734531,0.020672,0.325349,0.096454,0.59481,-0.051653,0.055888,0.427645
503,ZION,71.040,N/A,0.461176,0.802395,0.362315,0.96008,0.105308,0.61477,0.079699,0.738523,0.778942


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [14]:
hqm_df.sort_values("HQM Score", ascending=False, inplace=True)
hqm_df = hqm_df[:50]
hqm_df.reset_index(drop=True, inplace=True)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,F,24.920,N/A,1.831131,0.998004,0.72798,0.994012,0.764349,1.0,0.286221,1.0,0.998004
1,DVN,49.690,N/A,1.797481,0.996008,0.798526,0.996008,0.27917,0.96008,0.169777,0.984032,0.984032
2,MRO,18.580,N/A,1.474943,0.994012,0.382426,0.966068,0.202046,0.89022,0.160806,0.976048,0.956587
3,APA,30.410,N/A,0.542678,0.872255,0.456683,0.992016,0.345106,0.982036,0.149638,0.97006,0.954092
4,FANG,127.940,N/A,1.288205,0.99002,0.385254,0.968064,0.201943,0.888224,0.147397,0.968064,0.953593
5,AVGO,634.020,N/A,0.548837,0.874251,0.380221,0.964072,0.319568,0.97006,0.140799,0.956088,0.941118
6,ANET,133.480,N/A,0.88479,0.976048,0.417848,0.98004,0.512091,0.998004,0.092073,0.802395,0.939122
7,CF,69.870,N/A,0.663727,0.934132,0.414178,0.978044,0.163331,0.796407,0.179954,0.988024,0.924152
8,CMA,100.032,N/A,0.642772,0.922156,0.439134,0.986028,0.164983,0.806387,0.162911,0.978044,0.923154
9,AZO,2115.330,N/A,0.680349,0.944112,0.348675,0.956088,0.250923,0.946108,0.100553,0.840319,0.921657


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [15]:
portfolio_input()


Enter the size of your portfolio:25000


In [16]:
position_size = float(portfolio_size)/len(hqm_df.index)
for i in hqm_df.index:
    hqm_df.loc[i, "Number of Shares to Buy"] = math.floor(position_size/hqm_df.loc[i, 'Price'])
hqm_df

C:\Users\Local User\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,F,24.920,20,1.831131,0.998004,0.72798,0.994012,0.764349,1.0,0.286221,1.0,0.998004
1,DVN,49.690,10,1.797481,0.996008,0.798526,0.996008,0.27917,0.96008,0.169777,0.984032,0.984032
2,MRO,18.580,26,1.474943,0.994012,0.382426,0.966068,0.202046,0.89022,0.160806,0.976048,0.956587
3,APA,30.410,16,0.542678,0.872255,0.456683,0.992016,0.345106,0.982036,0.149638,0.97006,0.954092
4,FANG,127.940,3,1.288205,0.99002,0.385254,0.968064,0.201943,0.888224,0.147397,0.968064,0.953593
5,AVGO,634.020,0,0.548837,0.874251,0.380221,0.964072,0.319568,0.97006,0.140799,0.956088,0.941118
6,ANET,133.480,3,0.88479,0.976048,0.417848,0.98004,0.512091,0.998004,0.092073,0.802395,0.939122
7,CF,69.870,7,0.663727,0.934132,0.414178,0.978044,0.163331,0.796407,0.179954,0.988024,0.924152
8,CMA,100.032,4,0.642772,0.922156,0.439134,0.986028,0.164983,0.806387,0.162911,0.978044,0.923154
9,AZO,2115.330,0,0.680349,0.944112,0.348675,0.956088,0.250923,0.946108,0.100553,0.840319,0.921657


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [17]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_df.to_excel(writer, sheet_name = "Momentum Strategy", index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [18]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [19]:
columns_formats = {
    "A": ["Ticker", string_template],
    "B": ["Price", dollar_template],
    "C": ["Number of Shares to Buy", integer_template],
    "D": ["One-Year Price Return", percent_template],
    "E": ["One-Year Return Percentile", percent_template],
    "F": ["Six-Month Price Return", percent_template],
    "G": ["Six-Month Return Percentile", percent_template],
    "H": ["Three-Month Price Return", percent_template],
    "I": ["Three-Month Return Percentile", percent_template],
    "J": ["One-Month Price Return", percent_template],
    "K": ["One-Month Return Percentile", percent_template], 
    "L": ["HQM Score", percent_template]
}

for column in columns_formats.keys():
    writer.sheets["Momentum Strategy"].set_column(f'{column}:{column}', 22, columns_formats[column][1])
    
writer.save()


## Saving Our Excel Output

As before, saving our Excel output is very easy: